In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "notebooks"
    cd(joinpath(@__DIR__, "../"))
    Pkg.activate("Project.toml")
end

using MorphoMol

using CairoMakie
using LinearAlgebra
using Random
using Rotations
using StaticArrays
using Distributions
using Distances
using BenchmarkTools
using JLD2

  Activating project at `~/Desktop/Doktor/MorphoMol/MorphoMolMonteCarlo`


In [ ]:
file = "assets/input/jld2/large/6r7m_4_clumped.jld2"
@load file output_container

molecules = output_container.initial_state
mindex = argmin(output_container.Es)

for (i, step) in enumerate(output_container.steps)
    molecules.translations[step.index] += step.translation
    molecules.rotations[step.index] = step.rotation * molecules.rotations[step.index]
    if i == mindex
        break
    end
end

@save "four_close.jld2" molecules

In [109]:
@load "assets/input/jld2/2_6sag_protor_hardsphere_assembly.jld2" template_mol template_radii x_init
n_atoms_per_mol = length(template_mol) ÷ 3
n_mol = length(x_init) ÷ 6
template_mol = reshape(template_mol,(3,n_atoms_per_mol))
radii = vcat([template_radii for i in 1:n_mol]...);
MorphoMol.Utilities.state_to_poly(get_flat_realization(x_init, template_mol), radii, "assets/output/start", n_mol, n_atoms_per_mol)